In [2]:
import boto3
import pandas as pd
import io
from io import StringIO
import sagemaker
from sagemaker import get_execution_role

ModuleNotFoundError: No module named 'sagemaker'

In [2]:
#create sagemaker session and get execution role
sagemaker_session = sagemaker.Session()
role = get_execution_role()

In [3]:
#get training data from csv which is saved in your bucket
s3 = boto3.client('s3')
ion_data_obj = s3.get_object(Bucket = 'sunithadatasets', Key = 'data/train_data_with_features.csv')
ion_data = pd.read_csv(io.BytesIO(ion_data_obj['Body'].read()))

In [4]:
#get test data with features from csv which is saved in your bucket
test_data_obj = s3.get_object(Bucket = 'sunithadatasets',Key = 'data/test_data_with_features.csv')
test_data = pd.read_csv(io.BytesIO(test_data_obj['Body'].read()))

In [1]:
 ion_data.head()

NameError: name 'ion_data' is not defined

In [8]:
#used for predictions
test_values=test_data[['signal','simple_moving_avg_50','rolling_std_50','simple_moving_avg_50_batch','moving_std_50_batch']]

In [9]:
test_values.head()

,signal,simple_moving_avg_50,rolling_std_50,simple_moving_avg_50_batch,moving_std_50_batch
0,-2.6498,0.0,0.0,0.0,0.0
1,-2.8494,0.0,0.0,0.0,0.0
2,-2.8600,0.0,0.0,0.0,0.0
3,-2.4350,0.0,0.0,0.0,0.0
4,-2.6155,0.0,0.0,0.0,0.0


In [10]:
#get only columns which you want to feed into your model
X = ion_data[['open_channels','signal','simple_moving_avg_50','rolling_std_50','simple_moving_avg_50_batch','moving_std_50_batch']]


In [12]:
# Save this X (training data into a file X_train)
bucket ='sunithadatasets'
csv_buffer = StringIO()
X.to_csv(csv_buffer,index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,'data/X_train.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'CE5B92276D77085B',
  'HostId': 'sTqnXsSilcbjtYfAsYHA6g4wbRlFURcoeqWWmtTbJBCRoV1bDz5Obc8oIPfkk1WWeAMMD7tsHsM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'sTqnXsSilcbjtYfAsYHA6g4wbRlFURcoeqWWmtTbJBCRoV1bDz5Obc8oIPfkk1WWeAMMD7tsHsM=',
   'x-amz-request-id': 'CE5B92276D77085B',
   'date': 'Tue, 24 Mar 2020 15:35:19 GMT',
   'etag': '"3df72821963d14ae3be8888e0017ff40"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"3df72821963d14ae3be8888e0017ff40"'}

In [13]:
X.head()

,open_channels,signal,simple_moving_avg_50,rolling_std_50,simple_moving_avg_50_batch,moving_std_50_batch
0,0,-2.7600,0.0,0.0,0.0,0.0
1,0,-2.8557,0.0,0.0,0.0,0.0
2,0,-2.4074,0.0,0.0,0.0,0.0
3,0,-3.1404,0.0,0.0,0.0,0.0
4,0,-3.1525,0.0,0.0,0.0,0.0


In [14]:
#assign region,set client/service and bucket

region = boto3.Session().region_name
smclient = boto3.Session().client('sagemaker')

bucket = sagemaker.Session().default_bucket()


In [15]:
from sagemaker.amazon.amazon_estimator import get_image_uri

In [16]:
container = get_image_uri(region,'xgboost',repo_version='0.90-1')

In [17]:
#instantiating an sagemaker container where we run our models. speicifying the details.
xgb = sagemaker.estimator.Estimator(
container,
role,
train_instance_count =1,
#train_instance_type = 'ml.m4.xlarge',
  train_instance_type = 'ml.m5.4xlarge'
output_path='s3://{}/{}/output'.format('sunithadatasets','data'),
sagemaker_session = sagemaker_session)

In [18]:
xgb.set_hyperparameters(
num_round = 100,
rate_drop = 0.3,
alpha = 0.25,
num_class = 11,
objective = "multi:softmax"
)

In [19]:
#now we need to pass the x_data to the model. hence set the path
s3_input_train = sagemaker.s3_input(s3_data = 's3://{}/{}/X_train'.format('sunithadatasets','data'),content_type ='csv')

In [20]:
xgb.fit({'train' : s3_input_train})

2020-03-24 15:38:51 Starting - Starting the training job...
2020-03-24 15:38:52 Starting - Launching requested ML instances......
2020-03-24 15:40:17 Starting - Preparing the instances for training.........
2020-03-24 15:41:35 Downloading - Downloading input data...
2020-03-24 15:42:16 Training - Downloading the training image...
2020-03-24 15:42:30 Training - Training image download completed. Training in progress.INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[15:42:33] 5000001x5 matrix with 25000005 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
INFO:root:Single node training.
INFO:root:Train matrix has 5000001 rows
[15:42:33] WARNING: /workspace/src/learne

In [21]:
#now, after training the model, lets deploy
predictor = xgb.deploy(initial_instance_count =1, instance_type = "ml.m4.xlarge")

-------------!

In [22]:
import numpy as np
from sagemaker.predictor import csv_serializer

In [23]:
predictor.content_type = 'text/csv'
predictor.serializer = csv_serializer
predictor.deserializer = None

def predict(data, rows=500):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [24]:
y_pred = predict(test_values.to_numpy())

In [26]:
y_pred

array([-0.13467342, -0.13467342, -0.13467342, ...,  0.00069079,
        0.00069079,  0.00069079])

In [27]:
#convert array into a dataframe
df = pd.DataFrame(y_pred)

In [28]:
df

,0
0,-0.134673
1,-0.134673
2,-0.134673
3,-0.228436
4,-0.134673
5,-0.228436
6,-0.134673
7,-0.134673
8,-0.134673
9,0.119589


In [29]:
submission6 =pd.DataFrame()
submission6['time'] = test_data.time
submission6['open_channels'] = y_pred.astype(int)

submission6.time = submission6.time.apply(lambda x: '{:.4f}'.format(x))
#submission6.to_csv('data/XGBoost_submission5.csv',index=False)

In [32]:
submission6.head()

,time,open_channels
0,500.0001,0
1,500.0002,0
2,500.0003,0
3,500.0004,0
4,500.0005,0


In [33]:
# Save final kagggle file to csv
bucket ='sunithadatasets'
csv_buffer = StringIO()
submission6.to_csv(csv_buffer,index=False)
s3_resource = boto3.resource('s3')
s3_resource.Object(bucket,'data/XGBoost_submission6.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '2FA7E268B106E289',
  'HostId': 'RKr30ilnJYFYhUbxL9whb1NVyNVbsIoviI3fx8DjahGslpjysmoZve/fUhnU5GM7fc49dBRenqE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'RKr30ilnJYFYhUbxL9whb1NVyNVbsIoviI3fx8DjahGslpjysmoZve/fUhnU5GM7fc49dBRenqE=',
   'x-amz-request-id': '2FA7E268B106E289',
   'date': 'Tue, 24 Mar 2020 16:09:33 GMT',
   'etag': '"9b19b27af19733bedd7fac719646e810"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"9b19b27af19733bedd7fac719646e810"'}